# Machine learning homework7

## 0 Preparation

### 0.1 Import librarys

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cv2 import imread, IMREAD_GRAYSCALE

### 0.2 Read data from files

In [ ]:
mnist_x = np.loadtxt('data/mnist_X.csv', delimiter=',')
mnist_y = np.loadtxt('data/mnist_label.csv', delimiter=',')

## 1 Different dimension reduction methods

In [ ]:
def dim_reduce(x, y, method, need_y=False, title=''):
    """Use specific method to reduce dimension to 2 and plot result."""
    x_reduce = method(x, y, reduce_to=2) if need_y else method(x, reduce_to=2)
    fig, ax = plt.subplots()
    fig.suptitle(title)
    ax.scatter(x_reduce[:, 0], x_reduce[:, 1], c=y)
    plt.show()

### 1.1 PCA (Principal Component Analysis)

In [ ]:
def pca(x, reduce_to=2, return_eigen=False):
    """Perform PCA on dataset x, reduce to a certain dimension."""
    # Compute convariance matrix
    x_cov = np.cov(x, rowvar=False)
    # Compute eigen value and eigen vector of convariance
    eigen_val, eigen_vec = np.linalg.eig(x_cov)
    # Sort eigen value and vector according to eigen value in descending order
    eigen_vec = eigen_vec[:, np.argsort(eigen_val)[::-1]]
    eigen_val = eigen_val[np.argsort(eigen_val)[::-1]]
    # Project original data to new space
    result = x @ eigen_vec[:, :reduce_to]
    ret = result.real if not return_eigen else (result.real, (eigen_val, eigen_vec)) 
    return ret

In [ ]:
dim_reduce(mnist_x, mnist_y, pca, title='PCA without standardization')

In [ ]:
def standardize(x):
    """Standardize x, resulting in zero mean and unit variance."""
    z = np.zeros_like(x)
    std_x = x.std(axis=0)
    std_x_gt0 = std_x > 0
    z[:, std_x_gt0] = (x - x.mean(axis=0))[:, std_x_gt0] / std_x[std_x_gt0]
    return z

In [ ]:
dim_reduce(standardize(mnist_x), mnist_y, pca, title='PCA with standardization')

### 1.2 LDA (Linear Discriminant Analysis)

Reference:
- https://sebastianraschka.com/Articles/2014_python_lda.html

In [ ]:
def lda(x, y, reduce_to=2, return_eigen=False):
    """Perform LDA on dataset x, reduce to a certain dimension."""
    labels = np.unique(y)
    n_dim = x.shape[1]
    s_w = np.zeros((n_dim, n_dim))
    s_b = np.zeros_like(s_w)
    total_mean = x.mean(axis=0)
    for label in labels:
        mean = x[y==label].mean(axis=0)
        # Compute within-class scatter matrix
        s_w += np.cov(x[y==label], rowvar=False)
        # Compute between-class scatter matrix
        to_total_mean = mean - total_mean
        s_b += (to_total_mean[None, :] * to_total_mean[:, None]) * (y==label).sum()
    # Get eigen value and eigen vector from (s_w)^-1 * s_b
    eigen_val, eigen_vec = np.linalg.eig(np.linalg.pinv(s_w) @ s_b)
    # Sort eigen value and vector according to eigen value in descending order
    eigen_vec = eigen_vec[:, np.argsort(eigen_val)[::-1]]
    eigen_val = eigen_val[np.argsort(eigen_val)[::-1]]
    # Project original data to new space
    result = x @ eigen_vec[:, :reduce_to]
    ret = result.real if not return_eigen else (result.real, (eigen_val, eigen_vec)) 
    return ret

In [ ]:
dim_reduce(mnist_x, mnist_y, lda, need_y=True, title='LDA without standardization')

In [ ]:
dim_reduce(standardize(mnist_x), mnist_y, lda, need_y=True, title='LDA with standardization')

### 1.3 Symmetric SNE and T-SNE (Stochastic Neighbor Embedding)

## 2 Eigen face

### 2.1 Read all images

In [ ]:
n_subjects = 40
n_img_per_subject = 10
images = None
for i in range(n_subjects):
    for j in range(n_img_per_subject):
        path = f'data/att_faces/s{i + 1}/{j + 1}.pgm'
        img = imread(path, IMREAD_GRAYSCALE)[None]
        images = img if images is None else np.concatenate((images, img))
n_images, img_h, img_w = images.shape

### 2.2 Define normalization function

In [ ]:
def normalize(x):
    """Normalize x."""
    x_min = x.min(axis=0)
    x_max = x.max(axis=0)
    z = (x - x_min) / (x_max - x_min)
    return z

### 2.3 Apply PCA to normalized images

In [ ]:
def to_images(data, h, w, reverse=False):
    """Convert features to images, or reverse."""
    if reverse:  # Images to feature
        if data.ndim == 2:  # One data
            result = data.reshape(np.prod(data.shape))
        else:  # Multiple data
            result = data.reshape((data.shape[0], np.prod(data.shape[1:])))
    else:  # Features to images
        if data.ndim == 1:  # One data
            result = data.reshape((h, w))
        else:  # Multiple data
            result = data.reshape((data.shape[0], h, w))
    return result
images_x = to_images(images, None, None, reverse=True)

In [ ]:
n_pc = 32
images_x_reduced, eigen = pca(normalize(images_x), reduce_to=n_pc, return_eigen=True)

### 2.4 Show eigen face

In [ ]:
n_eigen_face = 25
n_col = 5
n_row = np.ceil(n_eigen_face / 5).astype(int)
fig, ax = plt.subplots(n_row , n_col, figsize=(n_col * 2.5, n_row * 2.5))
fig.suptitle(f'First {n_eigen_face} eigen face')
for i in range(n_eigen_face):
    row_idx = i // n_col
    col_idx = int(i % n_col)
    img = to_images(eigen[1][:, i].real, img_h, img_w)
    ax[row_idx, col_idx].set_title(f'{i + 1}-th eigen face')
    ax[row_idx, col_idx].imshow(img, cmap='gray')
    ax[row_idx, col_idx].set_axis_off()
plt.show()

### 2.5 Reconstruct faces

In [ ]:
n_reconstruct = 10
reconstruct_idx = np.random.randint(0, n_images, n_reconstruct)
fig, ax = plt.subplots(n_reconstruct , 2, figsize=(6, n_reconstruct * 2.5))
fig.suptitle(f'Random {n_reconstruct} face to reconstuct')
for i in range(n_reconstruct):
    idx = reconstruct_idx[i]
    ax[i, 0].imshow(images[idx], cmap='gray')
    ax[i, 0].set_title('original')
    ax[i, 0].set_axis_off()
    img = (images_x_reduced[idx] @ eigen[1][:, 0:n_pc].T).real
    img = to_images(img, img_h, img_w)
    ax[i, 1].imshow(img, cmap='gray')
    ax[i, 1].set_title('reconstruct')
    ax[i, 1].set_axis_off()
plt.show()